In [1]:
import sys
sys.path.append('./../../')

import ikamoana
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import xarray as xr

filepath_ika_config = "./../../data/ikamoana_config/IkaSim_Example.xml"
filepath_effort = "./../../data//SEAPODYM_data/run-test/data/fisheries/skj_fisheries_data_2021_Nov17.txt"

In [2]:
ika = ikamoana.IkamoanaFields(filepath_ika_config)
df_effort = ikamoana.fisherieseffort.NEWfisherieseffort.readFiles(filepath_effort, 0)
coords_time = ika.feeding_habitat_structure.data_structure.coords['time'].data
coords_lat = ika.feeding_habitat_structure.data_structure.coords['lat'].data
coords_lon = ika.feeding_habitat_structure.data_structure.coords['lon'].data

> Original sum of effort is :

In [3]:
df_effort['E'].sum()

402336382.99999994

## Reindex time coordinates

In [4]:
reindexed_df_effort = ikamoana.fisherieseffort.NEWfisherieseffort._reindexTime(
    df_effort, coords_time, limit=30)
reindexed_df_effort

,f,time,lat,lon,res,gr,E
0,1,1950-01-15,46.5,150.5,1,P,1.0
1,1,1950-01-15,34.5,136.5,1,P,0.0
2,1,1950-01-15,34.5,138.5,1,P,0.0
3,1,1950-01-15,34.5,139.5,1,P,9.0
4,1,1950-01-15,33.5,133.5,1,P,0.0
...,...,...,...,...,...,...,...
1182367,15,2014-08-15,11.5,214.5,5,P,3.0
1182368,15,2014-08-15,10.5,229.5,5,P,3.0
1182369,15,2014-08-15,9.5,233.5,5,P,3.0
1182370,15,2014-08-15,2.5,239.5,5,P,3.0


## Reindex space coordinates

In [5]:
reindexed_df_effort = ikamoana.fisherieseffort.NEWfisherieseffort._reindexSpace(
    reindexed_df_effort, coords_lat, coords_lon, limit=2)
reindexed_df_effort

,f,time,lat,lon,res,gr,E
0,1,1950-01-15,46.5,149.5,1,P,1.0
1,1,1950-01-15,34.5,135.5,1,P,0.0
2,1,1950-01-15,34.5,137.5,1,P,0.0
3,1,1950-01-15,34.5,139.5,1,P,9.0
4,1,1950-01-15,34.5,133.5,1,P,0.0
...,...,...,...,...,...,...,...
1182367,15,2014-08-15,12.5,213.5,5,P,3.0
1182368,15,2014-08-15,10.5,229.5,5,P,3.0
1182369,15,2014-08-15,10.5,233.5,5,P,3.0
1182370,15,2014-08-15,2.5,239.5,5,P,3.0


## Transforme into DataArray then calculate the sum of effort

In [6]:
da_effort = reindexed_df_effort[['time','lat','lon','E']].groupby(['time','lat','lon']).sum()
da_effort = da_effort.to_xarray()
da_effort = da_effort['E']
print(da_effort.sum().data)
da_effort

402336382.9999996


<xarray.DataArray 'E' (time: 857, lat: 51, lon: 91)>
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]])
Coordinates:
  * time     (time) datetime64[ns] 1950-01-15 1950-02-15 ... 2021-06-15
  * lat      (lat) float64 -49.5 -45.5 -43.5 -41.5 -39.5 ... 48.5 50.5 52.5 56.5
  * lon      (lon) float64 0.5 109.5 111.5 113.5 ... 281.5 283.5 285.5 287.5

> Nothing loosed

## Reindex like Feeding Habitat forcing

In [7]:
print(da_effort.reindex(ika.feeding_habitat_structure.data_structure.coords).sum().data)
da_effort.reindex(ika.feeding_habitat_structure.data_structure.coords)

221280185.2999999


<xarray.DataArray 'E' (time: 384, lat: 60, lon: 101)>
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]])
Coordinates:
  * time     (time) datetime64[ns] 1979-01-15 1979-02-14 ... 2010-12-16
  * lat      (lat) float32 64.5 62.5 60.5 58.5 56.5 ... -47.5 -49.5 -51.5 -53.5
  * lon      (lon) float32 89.5 91.5 93.5 95.5 97.5 ... 283.5 285.5 287.5 289.5

> Sum of effort is the same for manual localisation and automatic reindexing.

In [8]:
tmp = da_effort.loc[
    '1979-01-15':'2010-12-16',
    -53.5:64.5,
    89.5:289.5
]
print(tmp.sum().data)
tmp

221280185.29999992


<xarray.DataArray 'E' (time: 384, lat: 51, lon: 90)>
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]])
Coordinates:
  * time     (time) datetime64[ns] 1979-01-15 1979-02-14 ... 2010-12-16
  * lat      (lat) float64 -49.5 -45.5 -43.5 -41.5 -39.5 ... 48.5 50.5 52.5 56.5
  * lon      (lon) float64 109.5 111.5 113.5 115.5 ... 281.5 283.5 285.5 287.5

Finally, we performed a downscaling from 1 to 2 degrees.